In [9]:
!pip install dash-html-components
!pip install dash_core_components

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash 2.9.3 requires dash-core-components==2.0.0, which is not installed.
dash 2.9.3 requires dash-table==5.0.0, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash 2.9.3 requires dash-table==5.0.0, which is not installed.


In [12]:
import pandas as pd
import requests
from io import StringIO
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px


In [3]:
url  = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [6]:
resp = requests.get(url)
data = resp.text
dt = StringIO(data)
df = pd.read_csv(dt, delimiter = ',')

In [30]:
df.isnull().sum()

Unnamed: 0                  0
Flight Number               0
Launch Site                 0
class                       0
Payload Mass (kg)           0
Booster Version             0
Booster Version Category    0
dtype: int64

In [32]:
df.drop('Unnamed: 0' , axis = 1, inplace = True)

In [19]:
app = dash.Dash(__name__)

### TASK 1: Add a Launch Site Drop-down Input Component


We have four different launch sites and we would like to first see which one has the largest success count. Then,
we would like to select one specific site and check its detailed success rate (class=0 vs. class=1).

As such, we will need a dropdown menu to let us select different launch sites

In [58]:
app.layout =  html.Div([html.H1('Sample Dash'), 
                        dcc.Dropdown(id='site-dropdown', options=[
                           {'label': 'All Sites', 'value': 'All Sites'},
                           {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                           {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                           {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                           {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                        placeholder='Select a Launch Site',
                        value='All Sites',
                        searchable=True),
                        html.Br(),
                       
                       #TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
                       html.Div(dcc.Graph(id = 'success-pie-chart')),
                        
                        #Task 3: Now add a range slider
                        dcc.RangeSlider(id='payload-slider',
                                        min=0, max=10000, step=1000,
                                        marks={i: '{}'.format(i) for i in range(0,10001,1000)},
                                        value=[min, max]),
                       
                       #TASK4: Plot a scatter chart
                       html.Div(dcc.Graph(id='success-payload-scatter-chart'))])

In [59]:
# Function decorator to specify function input and output
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def get_pie_chart(launched):
    if launched == 'All Sites':
        fig = px.pie(
            values=spacex_df.groupby('Launch Site')['class'].mean(),
            names=spacex_df.groupby('Launch Site')['Launch Site'].first(),
            title='Total Success Launches by Site'
        )
    else:
        filtered_data = spacex_df[spacex_df['Launch Site'] == launched]
        success_counts = filtered_data['class'].value_counts(normalize=True)
        fig = px.pie(
            values=success_counts,
            names=success_counts.index,
            title=f'Total Success Launches for Site {launched}'
        )

    return fig


In [60]:
#Funtion to render scatter plot
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'), Input('payload-slider', 'value')]
)
def get_scatter_chart(launched,payload):
    if launched == 'All Sites':
        filtered_data = spacex_df
    else:
        filtered_data = spacex_df[spacex_df['Launch Site'] == launched]

    if payload is not None:
        filtered_data = filtered_data[
            (filtered_data['Payload Mass (kg)'] >= payload[0]) &
            (filtered_data['Payload Mass (kg)'] <= payload[1])
        ]

    fig = px.scatter(
        filtered_data,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title='Payload vs Outcome Scatter Plot'
    )

    return fig

In [55]:
df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [62]:
if __name__ == '__main__':
    app.run_server()